# FloPy 

### Model checker demonstration

In [1]:
import os
import flopy

#### Set the working directory

In [2]:
path = os.path.join('..', 'data', 'mf2005_test')

#### Load example dataset and change the model work space

In [3]:
m = flopy.modflow.Modflow.load('test1ss.nam', model_ws=path)
m.change_model_ws('data')


changing model workspace...
   data


By default, the checker performs a model-level check when a set of model files are loaded, unless load is called with `check=False`. The load check only produces screen output if load is called with `verbose=True`. Checks are also performed at the package level when an individual package is loaded

#### The `check()` method
Each model and each package has a `check()` method. The check method has three arguments:

In [4]:
help(m.check)

Help on method check in module flopy.mbase:

check(f=None, verbose=True, level=1) method of flopy.modflow.mf.Modflow instance
    Check model data for common errors.
    
    Parameters
    ----------
    f : str or file handle
        String defining file name or file handle for summary file
        of check method output. If a string is passed a file handle
        is created. If f is None, check method does not write
        results to a summary file. (default is None)
    verbose : bool
        Boolean flag used to determine if check method results are
        written to the screen
    level : int
        Check method analysis level. If level=0, summary checks are
        performed. If level=1, full checks are performed.
    
    Returns
    -------
    None
    
    Examples
    --------
    
    >>> import flopy
    >>> m = flopy.modflow.Modflow.load('model.nam')
    >>> m.check()



#### The check class

By default, check is called at the model level without a summary output file, but with `verbose=True` and `level=1`. The check methods return an instance of the **check** class, which is housed with the flopy utilities.

In [5]:
chk = m.check()


test1ss MODEL DATA VALIDATION SUMMARY:
  116 Errors:
    LPF package: zero or negative horizontal hydraulic conductivity values
  233 Warnings:
    LPF package: horizontal hydraulic conductivity values below checker threshold of 1e-11
    LPF package: vertical hydraulic conductivity values below checker threshold of 1e-11
    RCH package: Variable NRCHOP set to value other than 3

  Checks that passed:
    Compatible solver package
    Unit number conflicts
    LPF package: zero or negative vertical hydraulic conductivity values
    LPF package: negative horizontal anisotropy values
    LPF package: vertical hydraulic conductivity values above checker threshold of 100000.0
    LPF package: horizontal hydraulic conductivity values above checker threshold of 100000.0
    BAS6 package: isolated cells in ibound array
    BAS6 package: Not a number
    GHB package: BC indices valid
    GHB package: not a number (Nan) entries
    GHB package: BC in inactive cells
    GHB package: BC elevati

#### Summary array
Most of the attributes and methods in **check** are intended to be used by the ``check()`` methods. The central attribute of **check** is the summary array:

In [6]:
chk.summary_array

rec.array([ ('Error', 'LPF', 0, 0, 0, 0.0, '\r    LPF package: zero or negative horizontal hydraulic conductivity values'),
 ('Error', 'LPF', 0, 0, 1, 0.0, '\r    LPF package: zero or negative horizontal hydraulic conductivity values'),
 ('Error', 'LPF', 0, 1, 0, 0.0, '\r    LPF package: zero or negative horizontal hydraulic conductivity values'),
 ('Error', 'LPF', 0, 1, 1, 0.0, '\r    LPF package: zero or negative horizontal hydraulic conductivity values'),
 ('Error', 'LPF', 0, 1, 2, 0.0, '\r    LPF package: zero or negative horizontal hydraulic conductivity values'),
 ('Error', 'LPF', 0, 1, 5, 0.0, '\r    LPF package: zero or negative horizontal hydraulic conductivity values'),
 ('Error', 'LPF', 0, 1, 6, 0.0, '\r    LPF package: zero or negative horizontal hydraulic conductivity values'),
 ('Error', 'LPF', 0, 1, 7, 0.0, '\r    LPF package: zero or negative horizontal hydraulic conductivity values'),
 ('Error', 'LPF', 0, 2, 1, 0.0, '\r    LPF package: zero or negative horizontal hydra

This is a numpy record array that summarizes errors and warnings found by the checker. The package, layer-row-column location of the error, the offending value, and a description of the error are provided. In the checker, errors and warnings are loosely defined as follows:  
#### Errors:

either input that would cause MODFLOW to crash, or inputs that almost certainly mis-represent the intended conceptual model.

#### Warnings:

inputs that are potentially problematic, but may be intentional.

each package-level check produces a **check** instance with a summary array. The model level checks combine the summary arrays from the packages into a master summary array. At the model and the package levels, the summary array is used to generate the screen output shown above. At either level, the summary array can be written to a csv file by supply a filename to the `f` argument. Specifying `level=2` prints the summary array to the screen.

In [7]:
m.check(level=2)

Errors and/or Warnings encountered.
Errors and/or Warnings encountered.

test1ss MODEL DATA VALIDATION SUMMARY:
  116 Errors:
    LPF package: zero or negative horizontal hydraulic conductivity values
  233 Warnings:
    LPF package: horizontal hydraulic conductivity values below checker threshold of 1e-11
    LPF package: vertical hydraulic conductivity values below checker threshold of 1e-11
    RCH package: Variable NRCHOP set to value other than 3

  Checks that passed:
    Compatible solver package
    Unit number conflicts
    LPF package: zero or negative vertical hydraulic conductivity values
    LPF package: negative horizontal anisotropy values
    LPF package: vertical hydraulic conductivity values above checker threshold of 100000.0
    LPF package: horizontal hydraulic conductivity values above checker threshold of 100000.0
    BAS6 package: isolated cells in ibound array
    BAS6 package: Not a number
    GHB package: BC indices valid
    GHB package: not a number (Nan) e

#### example of package level check and summary file

In [8]:
m.rch.check()


RCH PACKAGE DATA VALIDATION:
  1 Warning:
    Variable NRCHOP set to value other than 3



#### example of summary output file

In [9]:
m.check(f='checksummary.csv')


test1ss MODEL DATA VALIDATION SUMMARY:
  116 Errors:
    LPF package: zero or negative horizontal hydraulic conductivity values
  233 Warnings:
    LPF package: horizontal hydraulic conductivity values below checker threshold of 1e-11
    LPF package: vertical hydraulic conductivity values below checker threshold of 1e-11
    RCH package: Variable NRCHOP set to value other than 3
  see data/checksummary.csv for details.

  Checks that passed:
    Compatible solver package
    Unit number conflicts
    LPF package: zero or negative vertical hydraulic conductivity values
    LPF package: negative horizontal anisotropy values
    LPF package: vertical hydraulic conductivity values above checker threshold of 100000.0
    LPF package: horizontal hydraulic conductivity values above checker threshold of 100000.0
    BAS6 package: isolated cells in ibound array
    BAS6 package: Not a number
    GHB package: BC indices valid
    GHB package: not a number (Nan) entries
    GHB package: BC in i

In [10]:
try:
    import pandas as pd
    df = pd.read_csv('data/checksummary.csv')
except:
    df = open('data/checksummary.csv').readlines()
df

,type,package,k,i,j,value,desc
0,Error,LPF,0,0,0,0,zero or negative horizontal hydraulic conducti...
1,Error,LPF,0,0,1,0,zero or negative horizontal hydraulic conducti...
2,Error,LPF,0,1,0,0,zero or negative horizontal hydraulic conducti...
3,Error,LPF,0,1,1,0,zero or negative horizontal hydraulic conducti...
4,Error,LPF,0,1,2,0,zero or negative horizontal hydraulic conducti...
5,Error,LPF,0,1,5,0,zero or negative horizontal hydraulic conducti...
6,Error,LPF,0,1,6,0,zero or negative horizontal hydraulic conducti...
7,Error,LPF,0,1,7,0,zero or negative horizontal hydraulic conducti...
8,Error,LPF,0,2,1,0,zero or negative horizontal hydraulic conducti...
9,Error,LPF,0,2,2,0,zero or negative horizontal hydraulic conducti...


#### checking on `write_input()`
checking is also performed by default when `write_input()` is called at the package or model level. Checking on write is performed with the same `verbose` setting as specified for the model. However, if errors or warnings are encountered and `level=1` (default) or higher, a screen message notifies the user of the errors.

By default, the checks performed on `load()` and `write_input()` save results to a summary file, which is named after the packge or the model.

In [11]:
m.write_input()